### 第2章: UNIXコマンド
> popular-names.txtは，アメリカで生まれた赤ちゃんの「名前」「性別」「人数」「年」をタブ区切り形式で格納したファイルである．以下の処理を行うプログラムを作成し，popular-names.txtを入力ファイルとして実行せよ．さらに，同様の処理をUNIXコマンドでも実行し，プログラムの実行結果を確認せよ．

In [163]:
!rm  ../data/popular-names.txt
!wget https://nlp100.github.io/data/popular-names.txt -P ../data

rm: cannot remove '../data/popular-names.txt': No such file or directory
--2024-10-31 16:45:13--  https://nlp100.github.io/data/popular-names.txt
Resolving nlp100.github.io (nlp100.github.io)... 185.199.108.153, 185.199.110.153, 185.199.111.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55026 (54K) [text/plain]
Saving to: ‘../data/popular-names.txt’

popular-names.txt   100%[===================>]  53.74K  --.-KB/s    in 0.008s  

2024-10-31 16:45:13 (6.63 MB/s) - ‘../data/popular-names.txt’ saved [55026/55026]



### 10. 行数のカウント
> 行数をカウントせよ．確認にはwcコマンドを用いよ．

In [164]:
import pandas as pd

path = '../data/popular-names.txt'
df_popular_names = pd.read_csv(path, sep="\t", header=None)

print("行数: ", df_popular_names.shape[0])

行数:  2780


In [165]:
!wc ../data/popular-names.txt

 2780 11120 55026 ../data/popular-names.txt


### 11. タブをスペースに置換
>タブ1文字につきスペース1文字に置換せよ．確認にはsedコマンド，trコマンド，もしくはexpandコマンドを用いよ．

In [166]:
with open(path, 'r') as raw_file:
     str_file = raw_file.read()
     str_file = str_file.replace('\t', ' ')

with open('../data/space-separated.txt', 'w') as file:
     file.write(str_file)

In [167]:
!cat ../data/popular-names.txt | tr "\t" " " > ../data/space-separated2.txt
!diff ../data/space-separated.txt ../data/space-separated2.txt

### 12. 1列目をcol1.txtに，2列目をcol2.txtに保存
> 各行の1列目だけを抜き出したものをcol1.txtに，2列目だけを抜き出したものをcol2.txtとしてファイルに保存せよ．確認にはcutコマンドを用いよ．

In [168]:
col1 = df_popular_names[0]
col2 = df_popular_names[1]

col1.to_csv('../data/col1.txt', sep=',', header=False, index=False)
col2.to_csv('../data/col2.txt', sep=',', header=False, index=False)

In [169]:
!cut -f 1 ../data/popular-names.txt > ../data/col1-cut.txt
!cut -f 2 ../data/popular-names.txt > ../data/col2-cut.txt

!diff ../data/col1.txt ../data/col1-cut.txt
!diff ../data/col2.txt ../data/col2-cut.txt

### 13. col1.txtとcol2.txtをマージ
> 12で作ったcol1.txtとcol2.txtを結合し，元のファイルの1列目と2列目をタブ区切りで並べたテキストファイルを作成せよ．確認にはpasteコマンドを用いよ．

In [170]:
col1col2 = pd.concat([col1, col2], axis=1)
col1col2.to_csv('../data/col1col2.txt', sep='\t', header=False, index=False)

In [171]:
!paste ../data/col1-cut.txt ../data/col2-cut.txt > ../data/col1col2-paste.txt
!diff ../data/col1col2.txt ../data/col1col2-paste.txt

### 14. 先頭からN行を出力
> 自然数Nをコマンドライン引数などの手段で受け取り，入力のうち先頭のN行だけを表示せよ．確認にはheadコマンドを用いよ．

In [172]:
n = int(input())
df_popular_names.head(n)

,0,1,2,3
0,Mary,F,7065,1880
1,Anna,F,2604,1880
2,Emma,F,2003,1880


In [173]:
!head -5 ../data/popular-names.txt

Mary	F	7065	1880
Anna	F	2604	1880
Emma	F	2003	1880
Elizabeth	F	1939	1880
Minnie	F	1746	1880


### 15. 末尾のN行を出力
> 自然数Nをコマンドライン引数などの手段で受け取り，入力のうち末尾のN行だけを表示せよ．確認にはtailコマンドを用いよ

In [174]:
n = int(input())
df_popular_names.tail(n)

,0,1,2,3
2777,Lucas,M,12585,2018
2778,Mason,M,12435,2018
2779,Logan,M,12352,2018


In [175]:
!tail -5 ../data/popular-names.txt

Benjamin	M	13381	2018
Elijah	M	12886	2018
Lucas	M	12585	2018
Mason	M	12435	2018
Logan	M	12352	2018


### 16. ファイルをN分割する
> 自然数Nをコマンドライン引数などの手段で受け取り，入力のファイルを行単位でN分割せよ．同様の処理をsplitコマンドで実現せよ．

In [176]:
total_partitions = int(input())
n_rows = df_popular_names.shape[0]

idx_start = 0 # 各分割の頭の位置
idx_end = n_rows // total_partitions # 各分割のおしりの位置

# 余りなく分割できるとき -> そのまま分割
if idx_end*total_partitions == n_rows:
    for i in range(total_partitions):

        output_path = f'../data/popular-names-part{i+1}'

        part = df_popular_names[idx_start:idx_end]
        part.to_csv(output_path, sep='\t', header=False, index=False)

        idx_start = idx_end
        idx_end += n_rows // total_partitions

# 余りが出るとき -> 剰余分を均一に割り振る
# 例: 11行を3分割 -> 4, 4, 3
else:
    reminder = n_rows % total_partitions
    for i in range(total_partitions):

        output_path = f'../data/popular-names-part{i+1}'
        idx_end += 1 if (reminder := reminder-1) >= 0 else 0 # 余剰分がなくなるまでidx_endを+1
        
        part = df_popular_names[idx_start:idx_end]
        part.to_csv(output_path, sep='\t', header=False, index=False)

        idx_start = idx_end
        idx_end += n_rows // total_partitions

In [177]:
!split -n 3 ../data/popular-names.txt ../data/

### 17. １列目の文字列の異なり
> 1列目の文字列の種類（異なる文字列の集合）を求めよ．確認にはcut, sort, uniqコマンドを用いよ

In [178]:
set(df_popular_names[0])

{'Abigail',
 'Aiden',
 'Alexander',
 'Alexis',
 'Alice',
 'Amanda',
 'Amelia',
 'Amy',
 'Andrew',
 'Angela',
 'Anna',
 'Annie',
 'Anthony',
 'Ashley',
 'Austin',
 'Ava',
 'Barbara',
 'Benjamin',
 'Bertha',
 'Bessie',
 'Betty',
 'Brandon',
 'Brian',
 'Brittany',
 'Carol',
 'Carolyn',
 'Charles',
 'Charlotte',
 'Chloe',
 'Christopher',
 'Clara',
 'Crystal',
 'Cynthia',
 'Daniel',
 'David',
 'Deborah',
 'Debra',
 'Donald',
 'Donna',
 'Doris',
 'Dorothy',
 'Edward',
 'Elijah',
 'Elizabeth',
 'Emily',
 'Emma',
 'Ethan',
 'Ethel',
 'Evelyn',
 'Florence',
 'Frances',
 'Frank',
 'Gary',
 'George',
 'Hannah',
 'Harper',
 'Harry',
 'Heather',
 'Helen',
 'Henry',
 'Ida',
 'Isabella',
 'Jacob',
 'James',
 'Jason',
 'Jayden',
 'Jeffrey',
 'Jennifer',
 'Jessica',
 'Joan',
 'John',
 'Joseph',
 'Joshua',
 'Judith',
 'Julie',
 'Justin',
 'Karen',
 'Kathleen',
 'Kelly',
 'Kimberly',
 'Larry',
 'Laura',
 'Lauren',
 'Liam',
 'Lillian',
 'Linda',
 'Lisa',
 'Logan',
 'Lori',
 'Lucas',
 'Madison',
 'Margaret

In [179]:
!cut -f 1 ../data/popular-names.txt | sort -u  > ../data/unique.txt

### 18. 各行を3コラム目の数値の降順にソート
> 各行を3コラム目の数値の逆順で整列せよ（注意: 各行の内容は変更せずに並び替えよ）．確認にはsortコマンドを用いよ（この問題はコマンドで実行した時の結果と合わなくてもよい）.

In [180]:
df_popular_names.sort_values(2, ascending=False)

,0,1,2,3
1340,Linda,F,99689,1947
1360,Linda,F,96211,1948
1350,James,M,94757,1947
1550,Michael,M,92704,1957
1351,Robert,M,91640,1947
...,...,...,...,...
27,Annie,F,1326,1881
28,Bertha,F,1324,1881
8,Bertha,F,1320,1880
29,Alice,F,1308,1881


In [181]:
!sort -r -k 3n -t \t ../data/popular-names.txt > ../data/popular-names-sorted.txt

### 19. 各行の1コラム目の文字列の出現頻度を求め，出現頻度の高い順に並べる
> 各行の1列目の文字列の出現頻度を求め，その高い順に並べて表示せよ．確認にはcut, uniq, sortコマンドを用いよ

In [182]:
df_freq = df_popular_names[0].value_counts()
df_freq.to_csv('../data/frequency1.txt', sep='\t', header=False, index=True)

In [184]:
!cut -f 1 ../data/popular-names.txt | sort | uniq -c | sort -r -k 1  > ../data/frequency2.txt